[Reference](https://levelup.gitconnected.com/sentiment-analysis-642b935ab6f9)

In [14]:
# # Install Kaggle library
# !pip install -q kaggle

# #upload the kaggle.json file you downloaded
# uploaded = files.upload()

# # make a diectoryin which kajggle.json is stored
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/

# #download the dataset into the colab(paste API command after !)
# !kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

# #unzip the data
# !unzip imdb-dataset-of-50k-movie-reviews.zip

In [15]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
!pwd
os.chdir('gdrive/My Drive/Colab Notebooks')
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content
/content/gdrive/My Drive/Colab Notebooks


In [16]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Bidirectional,Flatten,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Import and preprocess the data

In [17]:
def remove_special_chars(tweets): 
  # it unrolls the hashtags to normal words
 for remove in map(lambda r: re.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                        "@", "%", "^", "*", "(", ")", "{", "}",
                                                        "[", "]", "|", "/", "\\", ">", "<", "-",
                                                        "!", "?", ".", "'",
                                                        " — ", " — -", "#"]):
   tweets.replace(remove, "", inplace=True)
   return tweets

def remove_tags(text):
  return re.compile(r'<[^>]+>').sub('', text)

def remove_num(text):
  return ''.join(re.sub(r"([0–9]+)","",text))
 
data = pd.read_csv('IMDB Dataset.csv')
data.review = data.review.apply(lambda x : remove_tags(x))
data.review = data.review.apply(lambda x : remove_num(x))
remove_special_chars(data.review)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Word Embeddings

In [18]:
tokenizer = Tokenizer(num_words=5000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")

tokenizer.fit_on_texts(data['review'])
X = tokenizer.texts_to_sequences(data['review'])
X = pad_sequences(X,maxlen=500)
Y = data['sentiment']
vocab_size = len(tokenizer.word_index) + 1

# We can then create our train and test sets:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 24)

#We store this tokenizer in a file to use later in web app
import pickle
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
from sklearn.preprocessing import LabelEncoder
def prepare_targets(y_train, y_test):
  le = LabelEncoder()
  le.fit(y_train)
  y_train_enc = le.transform(y_train)
  y_test_enc = le.transform(y_test)
  return y_train_enc, y_test_enc
 
ytrain,ytest = prepare_targets(Y_train,Y_test)

# Build the model

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=500))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 50)           6585300   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               183296    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 6,768,853
Trainable params: 6,768,853
Non-trainable params: 0
_________________________________________________________________


In [20]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

history=model.fit(X_train, ytrain,
 batch_size=128,
 epochs=20,
 validation_data=[X_test, ytest],
 callbacks=[es])

#We save this model so that we can use in own web app
model.save('movie_sent.h5')

Epoch 1/20
274/274 [==============================] - 800s 3s/step - loss: 0.4611 - accuracy: 0.7727 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
274/274 [==============================] - 787s 3s/step - loss: 0.2740 - accuracy: 0.8933 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/20
274/274 [==============================] - 775s 3s/step - loss: 0.2393 - accuracy: 0.9079 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/20
274/274 [==============================] - 774s 3s/step - loss: 0.2147 - accuracy: 0.9176 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/20
274/274 [==============================] - 772s 3s/step - loss: 0.1874 - accuracy: 0.9289 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/20
274/274 [==============================] - 782s 3s/step - loss: 0.1946 - accuracy: 0.9253 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 00006: early stopping


In [21]:
string11='''Between the Lovecraftian overtones and Liberato’s performance, 
The Beach House offers up beautifully shot terror and will make you think before opening your door.'''
x_1=tokenizer.texts_to_sequences([string11])
x_1 = pad_sequences(x_1,maxlen=500)
model.predict(x_1)

array([[0.93133926]], dtype=float32)

In [22]:
!streamlit run sent_anaysis.py

/bin/bash: streamlit: command not found
